In [3]:
# import all libraries 
import keras 
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout 
from keras.layers.convolutional import Conv1D 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences 
import pandas as pd 
import numpy as np 
import spacy 
nlp=spacy.load("en")

In [4]:
#load the dataset 
train=pd.read_csv("training.1600000.processed.noemoticon.csv" , encoding= "latin-1") 
Y_train = train[train.columns[0]] 
X_train = train[train.columns[5]]

In [5]:
train.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [6]:
X_train.head()

0    is upset that he can't update his Facebook by ...
1    @Kenichan I dived many times for the ball. Man...
2      my whole body feels itchy and like its on fire 
3    @nationwideclass no, it's not behaving at all....
4                        @Kwesidei not the whole crew 
Name: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D, dtype: object

In [7]:
# split the data into test and train 
from sklearn.model_selection import train_test_split 
trainset1x, trainset2x, trainset1y, trainset2y = train_test_split(X_train.values, Y_train.values, test_size=0.02,random_state=42) 
trainset2y=pd.get_dummies(trainset2y) 

In [13]:
# function to remove stopwords 
def stopwords(sentence):   
    new=[]   
    sentence=nlp(sentence)
    for w in sentence:        
        if (w.is_stop == False) & (w.pos_ !="PUNCT"):            
            new.append(w.string.strip()) 
    c=" ".join(str(x) for x in new)    
    return c 

In [14]:
# function to lemmatize the tweets 
def lemmatize(sentence):    
    sentence=nlp(sentence)    
    str=""    
    for w in sentence:        
        str+=" "+w.lemma_    
    return nlp(str)

In [24]:
#loading the glove model 
def loadGloveModel(gloveFile):
    print("Loading Glove Model")    
    f = open(gloveFile,'r')    
    model = {}    
    for line in f:        
        splitLine = line.split()       
        word = splitLine[0]        
        embedding = [float(val) for val in splitLine[1:]]        
        model[word] = embedding    
    print (("Done."),len(model),(" words loaded!") )   
    return model 

In [25]:
# save the glove model 
model=loadGloveModel("glove.twitter.27B.200d.txt") 
#vectorising the sentences 
def sent_vectorizer(sent, model):    
    sent_vec = np.zeros(200)    
    numw = 0    
    for w in sent.split():        
        try:            
            sent_vec = np.add(sent_vec, model[str(w)])            
            numw+=1        
        except:            
            pass    
        return sent_vec 

Loading Glove Model
Done. 1193514  words loaded!


In [43]:
#obtain a clean vector 
cleanvector=[] 
for i in range(trainset2x.shape[0]):    
    document=trainset2x[i]    
document=document.lower()    
document=lemmatize(document)    
document=str(document)    
cleanvector.append(sent_vectorizer(document,model)) 

In [44]:
#Getting the input and output in proper shape 
cleanvector=np.array(cleanvector) 
cleanvector =cleanvector.reshape(80,200,-1) 

ValueError: cannot reshape array of size 200 into shape (80,200,newaxis)

In [45]:
cleanvector.shape

(1, 200)